To build/rebuild run: 
```
jbuilder build src-core/core.cma
jbuilder build src-core-pp/core_pp.cma
```

In [3]:
#use "topfind";;
#require "str";;
#require "yojson";;

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()
/home/kanishev/.opam/4.04.1/lib/ocaml/str.cma: loaded
/home/kanishev/.opam/4.04.1/lib/easy-format: added to search path
/home/kanishev/.opam/4.04.1/lib/easy-format/easy_format.cmo: loaded
/home/kanishev/.opam/4.04.1/lib/biniou: added to search path
/home/kanishev/.opam/4.04.1/lib/biniou/biniou.cma: loaded
/home/kanishev/.opam/4.04.1/lib/yojson: added to search path
/home/kanishev/.opam/4.04.1/lib/yojson/yojson.cmo: loaded


In [4]:
#directory "_build/default/src-core/";;
#directory "_build/default/src-core-pp/";;
#load "_build/default/src-core/core.cma";;
#load "_build/default/src-core-pp/core_pp.cma";;

In [5]:
let message_of_string str = str
    |> Stream.of_string 
    |> Parser_utils.split_into_key_value '|'
    |> Parser_utils.split_into_messages
    |> Stream.peek
    |> function Some m -> m | _ -> failwith "WAT"

val message_of_string : string -> (string * string) list = <fun>


In [6]:
let message = 
    "8=FIX.4.2^A9=182^A35=D^A34=200^A49=00000543^A56=EURONEXT^A115=00000543^A" ^
    "142=TESTTOOL^A52=20170824-13:45:58^A43=N^A97=N^A11=O1^A55=BE0947727377^A" ^
    "54=1^A40=2^A59=0^A44=8.5^A38=100^A60=20170824-13:46:15^A" ^
    "47=1^A57=EURONEXT^A10=221^A" 
    |> Str.(global_replace (regexp "\^A") "|" )
    |> message_of_string

File "", line 6, characters 36-38:
Warning 14: illegal backslash escape in string.
val message : (string * string) list =
  [("8", "FIX.4.2"); ("9", "182"); ("35", "D"); ("34", "200");
   ("49", "00000543"); ("56", "EURONEXT"); ("115", "00000543");
   ("142", "TESTTOOL"); ("52", "20170824-13:45:58"); ("43", "N");
   ("97", "N"); ("11", "O1"); ("55", "BE0947727377"); ("54", "1");
   ("40", "2"); ("59", "0"); ("44", "8.5"); ("38", "100");
   ("60", "20170824-13:46:15"); ("47", "1"); ("57", "EURONEXT");
   ("10", "221")]


In [7]:
let checksum s = 
    let rec chsum i v =
        if i < 0 then v else chsum (i - 1) (Char.code s.[i] + v)  in
    chsum (String.length s - 1) 0 
    in
    let rec scan n msg = match msg with
        | ("10", v )::tl -> ( n mod 256 , v )
        | (  k , v )::tl -> scan (checksum k + checksum v + 62 + n) tl
        | [] -> ( n mod 256 , "") in 
    scan 0 message
;;
let body_length =
    let rec scan n msg = match msg with
        | ( "8", _ )::tl
        | ( "9", _ )::tl -> scan n tl
        | ("10", _ )::tl -> n
        | (  k , v )::tl -> n + scan String.(length k + length v + 2) tl
        | [] -> n in
    scan 0 message
;;

- : int * string = (221, "221")
val body_length : int = 182


In [9]:
#directory "_build/default/src-protocol/";;
#directory "_build/default/src-protocol-exts/";;
#directory "_build/default/src-protocol-pp/";;
#directory "_build/default/src-protocol-exts-pp/";;

#load "_build/default/src-protocol-exts/protocol_exts.cma";;
#load "_build/default/src-protocol/protocol.cma";;
#load "_build/default/src-protocol-exts-pp/protocol_exts_pp.cma";;
#load "_build/default/src-protocol-pp/protocol_pp.cma";;

In [16]:
message 
|> Parse_full_messages.parse_top_level_msg 
|> Full_messages_json.full_top_level_msg_to_json
|> Yojson.pretty_to_string
|> print_endline

{
  "SessionRejectedMsg": {
    "RefSeqNum": 200,
    "RefTagID": "HandlInst",
    "RefMsgType": "OrderSingle",
    "SessionRejectReason": "RequiredTagMissing"
  }
}
- : unit = ()
